In [43]:
import random
import math
import pygame
import time

In [44]:
#total ingredient lists
all_fruits = ["banana", "apple", "blueberry", "strawberry",
              "raspberry", "mango", "orange", "pineapple",
              "kiwi", "mixed berries", "watermelon", "cherry", "blackberry"]
all_liquids = ["cow's milk", "soy milk", "almond milk", "coconut milk",
               "oat milk", "rice milk", "hazelnut milk", "skim milk",
               "natural yoghurt", "apple juice", "orange juice", "lemon juice",
               "cranberry juice", "pineapple juice", "grape juice", "pomegranate juice"]
all_miscs = ["honey", "maple syrup", "oats", "peanut butter",
             "almond butter", "protein powder", "mint", "cinnamon", "ginger", 
             "tumeric"]

#dietary restrictions
non_vegan = ["cow's milk", "skim milk", "natural yoghurt", "honey"]
dairy = ["cow's milk", "skim milk", "natural yoghurt"]

#starting ingredient lists
fruits = ["banana", "apple", "blueberry", "strawberry"]
liquids = ["cow's milk", "soy milk", "apple juice", "lemon juice"]
miscs = ["honey", "oats", "protein powder", "peanut butter"]

#starting max fruits
max_fruits = 2
#starting max liquids
max_liquids = 1
#starting max other ingredients
max_miscs = 1

In [45]:
#create order with up to 2 fruits, 1 liquid, 1 other
def create_order():
    order_ingredients = []
    #random liquid
    for liquid in random.sample(liquids, random.randint(1, max_liquids)):
        order_ingredients.append(liquid)
    #random misc
    for misc in random.sample(miscs, random.randint(1, max_miscs)):
        order_ingredients.append(misc)
    #get multiple fruits
    for fruit in random.sample(fruits, random.randint(1, max_fruits)):
        order_ingredients.append(fruit)
    order_ingredients.append("ice")

    return order_ingredients
    
create_order()

['soy milk', 'oats', 'strawberry', 'ice']

In [46]:
# initialize pygame
pygame.init()

#screen setup
WIDTH = 960
HEIGHT = 540

#create screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))

#create clock
clock = pygame.time.Clock

#title & icon
pygame.display.set_caption("Smoothie Shop")
icon = pygame.image.load("art/smoothie.png")
pygame.display.set_icon(icon)

#font
pygame.font.init()
main_font = pygame.font.SysFont('Kristen ITC', 28)
time_font = pygame.font.SysFont('LT Stopwatch', 60)

#load background images
bar = pygame.image.load("art/bar.png").convert_alpha()
prep_area = pygame.image.load("art/prep_area2.png").convert_alpha()

#load blender images
blender = pygame.image.load("art/blender.png").convert_alpha()
mixture = pygame.image.load("art/blender_contents.png").convert_alpha()

#load button images
kitchen_button_img = pygame.image.load("art/kitchen_button.png").convert_alpha()
service_button_img = pygame.image.load("art/service_button.png").convert_alpha()

#ingredient button images
#shelf ingredients
skim_milk_img = pygame.image.load("art/skim_milk.png").convert_alpha()
soy_milk_img = pygame.image.load("art/soy_milk.png").convert_alpha()
apple_juice_img = pygame.image.load("art/apple_juice.png").convert_alpha()
lemon_juice_img = pygame.image.load("art/lemon_juice.png").convert_alpha()

#gastronorm ingredients
ice_img = pygame.image.load("art/ice_gastro.png").convert_alpha()
banana_gastro_img = pygame.image.load("art/banana_gastro.png").convert_alpha()
apple_gastro_img = pygame.image.load("art/apple_gastro.png").convert_alpha()
blueberry_gastro_img = pygame.image.load("art/blueberry_gastro.png").convert_alpha()
strawberry_gastro_img = pygame.image.load("art/strawberry_gastro.png").convert_alpha()

#list of smoothies
smoothies = []
available_ingredients = []

#button class
class Button():
    def __init__(self, x, y, image, scale):
          width = image.get_width()
          height = image.get_height()
          self.image = pygame.transform.scale(image, (int(width * scale), int(height * scale)))
          self.rect = self.image.get_rect()
          self.rect.topleft = (x, y)
          self.clicked = False

    def draw(self):
        action = False
         
        #get mouse position
        pos = pygame.mouse.get_pos()

         #check mouseover and clicked conditions
        if self.rect.collidepoint(pos):
            if pygame.mouse.get_pressed()[0] == 1 and self.clicked == False:
                self.clicked = True
                action = True
        if pygame.mouse.get_pressed()[0] == 0:
            self.clicked = False

         #draw button onto screen
        screen.blit(self.image, (self.rect.x, self.rect.y))
        
        return action

#create button instances
kitchen_button = Button(50, HEIGHT-125, kitchen_button_img, 4)
service_button = Button(WIDTH-178, 50, service_button_img, 4)

#constant y coordinate for certain items
gastro_top_row = 152
gastro_mid_row = 218
gastro_bot_row = 284
top_shelf = 360
bot_shelf = 450

#list of clicked ingredients
clicked_ingredients = []

#ingredient buttons
#gastro ingredient buttons
ice_button = Button(214, gastro_bot_row, ice_img, 1)
banana_gastro_button = Button(214, gastro_top_row, banana_gastro_img, 1)
apple_gastro_button = Button(214, gastro_mid_row, apple_gastro_img, 1)
blueberry_gastro_button = Button(280, gastro_top_row, blueberry_gastro_img, 1)
strawberry_gastro_button = Button(280, gastro_mid_row, strawberry_gastro_img, 1)

#shelf ingredient buttons
skim_milk_button = Button(225, top_shelf, skim_milk_img, 1)
soy_milk_button = Button(225, bot_shelf, soy_milk_img, 1)

#starting list of buttons - add to each time ingredient unlocked
available_ingredients = [ice_button, banana_gastro_button, apple_gastro_button, blueberry_gastro_button,
                         strawberry_gastro_button, skim_milk_button, soy_milk_button]

#function for ingredient button presses - add ingredients to list of currently clicked ones
def ingred_click(button, ingred_name):
    if button.draw():
        clicked_ingredients.append(ingred_name)
        print(clicked_ingredients)

#game areas
game_areas = {"bar": 1, "kitchen": 2}
current_area = 1

# -------------------------------------------------------------------------------------------------
# ---------------------------------------game loop-------------------------------------------------
# -------------------------------------------------------------------------------------------------

#game loop
running = True
while running:

    # screen.fill((255, 120, 180)) #fill pink
    screen.blit(bar, (0,0))
    
    # clicking the area buttons
    if current_area == 1 and kitchen_button.draw():
         current_area = 2
    elif current_area == 2 and service_button.draw():
         current_area = 1

    # what happens in each area
    if current_area == 1:
         #at bar/service counter
         screen.blit(bar, (0, 0))
         kitchen_button.draw()
    elif current_area == 2:
        #in kitchen/prep area
        screen.blit(prep_area, (0, 0))
        service_button.draw()
        screen.blit(blender, (20, 20))
        #ingredient buttons
        ingred_click(skim_milk_button, "skim milk")
        skim_milk_button.draw()
        soy_milk_button.draw()
        banana_gastro_button.draw()
        apple_gastro_button.draw()
        blueberry_gastro_button.draw()
        strawberry_gastro_button.draw()
        if ice_button.draw():
             clicked_ingredients.append("ice")
             print(clicked_ingredients)

    for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.display.update()

pygame.quit()

['ice']
['ice', 'skim milk']
['ice', 'skim milk', 'skim milk']
['ice', 'skim milk', 'skim milk', 'ice']
['ice', 'skim milk', 'skim milk', 'ice', 'ice']
